###### Imports and Settings

In [274]:
import pandas as pd
import numpy as np
import requests
#import io
import pickle
from collections import deque
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [275]:
def percent(x, y):
    return (x/y)*100

In [276]:
data = pd.read_csv('../../data downloads/2020PL_StateDataCenter_Block.csv')
data.head()

,GEO_ID,ALAND,pop,raceeth_onerace,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces,raceeth_hispanicorlatino,raceeth_nothispanicorlatino,raceeth_whitealone_nothispanicorlatino,units_allhousing,occupancy_occupiedunits,occupancy_vacantunits,gq_total_series,gq_institutionalized,gq_institutionalized_correctionalfacilityadults,gq_institutionalized_juvenilefacility,gq_institutionalized_nursingorskillednursingfacility,gq_institutionalized_other,gq_noninstitutionalized,gq_noninstitutionalized_collegeoruniversitystudenthousing,gq_noninstitutionalized_militaryquarters,gq_noninstitutionalized_other
0,7500000US470110115022011,237488,19,16,16,0,0,0,0,0,3,0,19,16,8,8,0,0,0,0,0,0,0,0,0,0,0
1,7500000US470110115022012,563802,22,21,20,0,1,0,0,0,1,0,22,20,7,7,0,0,0,0,0,0,0,0,0,0,0
2,7500000US470110115022013,384718,39,39,38,0,0,0,0,1,0,1,38,38,14,14,0,0,0,0,0,0,0,0,0,0,0
3,7500000US470110115022014,1389860,176,173,173,0,0,0,0,0,3,0,176,173,78,76,2,0,0,0,0,0,0,0,0,0,0
4,7500000US470110115022015,45842,9,8,8,0,0,0,0,0,1,0,9,8,5,5,0,0,0,0,0,0,0,0,0,0,0


In [277]:
# cross = pd.read_csv('../../data downloads/block_place_crosswalk.csv')
cross = pd.read_csv('../../data downloads/newcrosswalk.csv')
cross = pd.read_csv('../../data downloads/newnewcrosswalk.csv')
cross.head()

,BLOCK,PLACE,COUNTY,GEO_ID
0,1039,Adams city,Robertson,7500000US471470802001039
1,1022,Adams city,Robertson,7500000US471470802001022
2,1020,Adams city,Robertson,7500000US471470802001020
3,1030,Adams city,Robertson,7500000US471470802001030
4,1027,Adams city,Robertson,7500000US471470802001027


In [278]:
df = data.merge(cross, on = 'GEO_ID')

In [279]:
df['PL_CO'] = df['PLACE'] + ',' + df['COUNTY']

In [280]:
df = df.set_index(['PLACE', 'COUNTY', 'GEO_ID', 'BLOCK'])

In [281]:
data = df.groupby(by = 'PL_CO').sum()

In [282]:
data.head()

,ALAND,pop,raceeth_onerace,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces,raceeth_hispanicorlatino,raceeth_nothispanicorlatino,raceeth_whitealone_nothispanicorlatino,units_allhousing,occupancy_occupiedunits,occupancy_vacantunits,gq_total_series,gq_institutionalized,gq_institutionalized_correctionalfacilityadults,gq_institutionalized_juvenilefacility,gq_institutionalized_nursingorskillednursingfacility,gq_institutionalized_other,gq_noninstitutionalized,gq_noninstitutionalized_collegeoruniversitystudenthousing,gq_noninstitutionalized_militaryquarters,gq_noninstitutionalized_other
PL_CO,,,,,,,,,,,,,,,,,,,,,,,,,,
"Adams city,Robertson",6191502,624,595,553,22,7,2,0,11,29,16,608,550,250,232,18,19,19,0,0,19,0,0,0,0,0
"Ashland City town,Cheatham",27630180,5193,4852,4261,310,17,33,11,220,341,426,4767,4175,2311,2145,166,235,146,80,0,66,0,89,0,0,89
"Belle Meade city,Davidson",7951661,2901,2808,2773,6,1,23,0,5,93,52,2849,2758,1130,1073,57,0,0,0,0,0,0,0,0,0,0
"Berry Hill city,Davidson",2345866,2112,1973,1747,115,12,66,0,33,139,123,1989,1700,1657,1446,211,3,0,0,0,0,0,3,0,0,3
"Bethpage CDP,Sumner",3440295,313,304,291,6,1,2,0,4,9,21,292,281,127,109,18,0,0,0,0,0,0,0,0,0,0


In [283]:
data = data.reset_index()

In [286]:
thing = data['PL_CO'].str.split(pat = ',', expand = True)

In [287]:
data['PLACE'] = thing[0]
data['COUNTY'] = thing[1]

In [288]:
data = data.drop(columns = ['PLACE', 'PL_CO'])

In [289]:
inc = data.groupby(['COUNTY']).sum().reset_index().rename(columns = {'COUNTY':'NAME'})
inc['NAME'] = inc['NAME'] + ' Incorporated'
inc

,NAME,ALAND,pop,raceeth_onerace,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces,raceeth_hispanicorlatino,raceeth_nothispanicorlatino,raceeth_whitealone_nothispanicorlatino,units_allhousing,occupancy_occupiedunits,occupancy_vacantunits,gq_total_series,gq_institutionalized,gq_institutionalized_correctionalfacilityadults,gq_institutionalized_juvenilefacility,gq_institutionalized_nursingorskillednursingfacility,gq_institutionalized_other,gq_noninstitutionalized,gq_noninstitutionalized_collegeoruniversitystudenthousing,gq_noninstitutionalized_militaryquarters,gq_noninstitutionalized_other
0,Cheatham Incorporated,104870265,14896,14072,13056,497,43,97,15,364,824,855,14041,12803,6235,5852,383,235,146,80,0,66,0,89,0,0,89
1,Davidson Incorporated,1304564434,715884,662109,400622,173092,3942,27869,354,56230,53775,98119,617765,386835,328309,301236,27073,24801,7230,4378,156,2497,199,17571,15112,0,2459
2,Dickson Incorporated,93945769,23693,22079,19624,1419,136,200,4,696,1614,1544,22149,19295,10039,9516,523,708,582,294,0,105,183,126,0,0,126
3,Houston Incorporated,19803230,2556,2445,2292,104,11,4,6,28,111,68,2488,2272,1133,1017,116,143,135,0,0,135,0,8,0,0,8
4,Humphreys Incorporated,40666514,7744,7311,6866,320,24,38,0,63,433,216,7528,6778,3512,3168,344,177,161,78,0,83,0,16,0,0,16
5,Maury Incorporated,161717566,63122,58292,44410,9967,297,803,36,2779,4830,5949,57173,43196,27783,25601,2182,1272,1078,298,87,612,81,194,0,0,194
6,Montgomery Incorporated,257446809,166722,148495,94978,40657,914,4143,867,6936,18227,19671,147051,89596,65752,61128,4624,4296,1699,1084,32,583,0,2597,1450,962,185
7,Robertson Incorporated,216151995,43499,40294,32060,4604,250,352,16,3012,3205,5514,37985,31027,16982,16163,819,773,690,477,0,213,0,83,0,0,83
8,Rutherford Incorporated,361755759,251362,230261,152339,48728,1416,10459,207,17112,21101,32736,218626,146467,98626,93117,5509,5516,1658,774,43,762,79,3858,2831,0,1027
9,Stewart Incorporated,27328670,2422,2323,2204,72,5,10,0,32,99,65,2357,2183,1046,925,121,235,235,167,0,68,0,0,0,0,0


In [290]:
inc['Square Acres'] = inc['ALAND']*0.000247105
inc['Square Miles'] = inc['ALAND']*0.00000038610215855
inc = inc.drop(columns = 'ALAND')

# Create Unincorporated Datapoints  
Bring in full counties, append, then subtract

In [291]:
tiger = pd.read_csv('../../outputs to join prefilter/TIDYTIGERS2020.csv')
tiger.head()

,NAME,GEO_ID,Square Acres,Square Miles
0,"Franklin city, Kentucky",1600000US2128918,9437.749960,14.746507
1,"Scottsville city, Kentucky",1600000US2169114,4033.776862,6.302786
2,"Mount Juliet city, Tennessee",1600000US4750780,16041.182095,25.064386
3,"Clarksville city, Tennessee",1600000US4715160,63616.393738,99.400769
4,"Dover city, Tennessee",1600000US4721400,2448.762859,3.826198


In [292]:
fullcounties = pd.read_csv('../../pull and download formatting outputs/PL2020.csv')
fullcounties = fullcounties.drop(columns = ['GEO_ID', 'StateFIPS', 'GeoFIPS']).set_index('NAME').transpose()
fullcounties = fullcounties[['Montgomery County, Tennessee', 'Rutherford County, Tennessee', 'Sumner County, Tennessee', 
                              'Trousdale County, Tennessee', 'Williamson County, Tennessee', 'Wilson County, Tennessee', 
                              'Cheatham County, Tennessee', 'Robertson County, Tennessee', 'Stewart County, Tennessee', 
                              'Davidson County, Tennessee', 'Dickson County, Tennessee', 'Houston County, Tennessee', 
                              'Humphreys County, Tennessee', 'Maury County, Tennessee']].transpose().reset_index()
fullcounties.head()

,NAME,pop,raceeth_total_series,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,gq_total_series,gq_institutionalized,gq_institutionalized_correctionalfacilityadults,gq_institutionalized_juvenilefacility,gq_institutionalized_nursingorskillednursingfacility,gq_institutionalized_other,gq_noninstitutionalized,gq_noninstitutionalized_collegeoruniversitystudenthousing,gq_noninstitutionalized_militaryquarters,gq_noninstitutionalized_other
0,"Montgomery County, Tennessee",220069.0,220069.0,137775.0,44569.0,1154.0,5202.0,1000.0,7890.0,22479.0,131294.0,22934.0,85714.0,85714.0,79841.0,5873.0,4714.0,1758.0,1084.0,32.0,642.0,0.0,2956.0,1450.0,1321.0,185.0
1,"Rutherford County, Tennessee",341486.0,341486.0,226227.0,53977.0,1739.0,12538.0,254.0,19755.0,26996.0,219194.0,38317.0,131216.0,131216.0,124465.0,6751.0,5723.0,1851.0,774.0,43.0,955.0,79.0,3872.0,2831.0,0.0,1041.0
2,"Sumner County, Tennessee",196281.0,196281.0,158075.0,15740.0,711.0,2956.0,119.0,5321.0,13359.0,155169.0,12870.0,78995.0,78995.0,74472.0,4523.0,1612.0,1377.0,572.0,25.0,780.0,0.0,235.0,0.0,0.0,235.0
3,"Trousdale County, Tennessee",11615.0,11615.0,8883.0,1922.0,30.0,27.0,0.0,205.0,548.0,8786.0,375.0,3749.0,3749.0,3469.0,280.0,2569.0,2549.0,2505.0,0.0,44.0,0.0,20.0,0.0,0.0,20.0
4,"Williamson County, Tennessee",247726.0,247726.0,203600.0,9876.0,574.0,12953.0,117.0,5172.0,15434.0,200408.0,14261.0,91133.0,91133.0,86884.0,4249.0,1359.0,1252.0,189.0,28.0,1035.0,0.0,107.0,0.0,0.0,107.0


In [293]:
counties = fullcounties.merge(tiger, on = 'NAME', how = 'inner')
counties = counties.drop(columns = 'GEO_ID')
counties

,NAME,pop,raceeth_total_series,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,gq_total_series,gq_institutionalized,gq_institutionalized_correctionalfacilityadults,gq_institutionalized_juvenilefacility,gq_institutionalized_nursingorskillednursingfacility,gq_institutionalized_other,gq_noninstitutionalized,gq_noninstitutionalized_collegeoruniversitystudenthousing,gq_noninstitutionalized_militaryquarters,gq_noninstitutionalized_other,Square Acres,Square Miles
0,"Montgomery County, Tennessee",220069.0,220069.0,137775.0,44569.0,1154.0,5202.0,1000.0,7890.0,22479.0,131294.0,22934.0,85714.0,85714.0,79841.0,5873.0,4714.0,1758.0,1084.0,32.0,642.0,0.0,2956.0,1450.0,1321.0,185.0,345067.747777,539.169188
1,"Rutherford County, Tennessee",341486.0,341486.0,226227.0,53977.0,1739.0,12538.0,254.0,19755.0,26996.0,219194.0,38317.0,131216.0,131216.0,124465.0,6751.0,5723.0,1851.0,774.0,43.0,955.0,79.0,3872.0,2831.0,0.0,1041.0,396329.938251,619.266485
2,"Sumner County, Tennessee",196281.0,196281.0,158075.0,15740.0,711.0,2956.0,119.0,5321.0,13359.0,155169.0,12870.0,78995.0,78995.0,74472.0,4523.0,1612.0,1377.0,572.0,25.0,780.0,0.0,235.0,0.0,0.0,235.0,338853.333043,529.459150
3,"Trousdale County, Tennessee",11615.0,11615.0,8883.0,1922.0,30.0,27.0,0.0,205.0,548.0,8786.0,375.0,3749.0,3749.0,3469.0,280.0,2569.0,2549.0,2505.0,0.0,44.0,0.0,20.0,0.0,0.0,20.0,73178.560819,114.341678
4,"Williamson County, Tennessee",247726.0,247726.0,203600.0,9876.0,574.0,12953.0,117.0,5172.0,15434.0,200408.0,14261.0,91133.0,91133.0,86884.0,4249.0,1359.0,1252.0,189.0,28.0,1035.0,0.0,107.0,0.0,0.0,107.0,373031.486909,582.862598
5,"Wilson County, Tennessee",147737.0,147737.0,120782.0,10191.0,601.0,3106.0,44.0,3827.0,9186.0,118889.0,8375.0,58152.0,58152.0,55074.0,3078.0,1866.0,1057.0,381.0,17.0,659.0,0.0,809.0,477.0,0.0,332.0,365356.705347,570.870733
6,"Cheatham County, Tennessee",41072.0,41072.0,36788.0,819.0,135.0,191.0,26.0,780.0,2333.0,36299.0,1839.0,16785.0,16785.0,15706.0,1079.0,349.0,224.0,80.0,0.0,144.0,0.0,125.0,0.0,0.0,125.0,193601.852087,302.503361
7,"Robertson County, Tennessee",72803.0,72803.0,58467.0,5168.0,335.0,460.0,21.0,3664.0,4688.0,57049.0,6878.0,28459.0,28459.0,26885.0,1574.0,807.0,690.0,477.0,0.0,213.0,0.0,117.0,0.0,0.0,117.0,304854.389113,476.335718
8,"Stewart County, Tennessee",13657.0,13657.0,12463.0,172.0,78.0,77.0,2.0,122.0,743.0,12321.0,336.0,6669.0,6669.0,5470.0,1199.0,235.0,235.0,167.0,0.0,68.0,0.0,0.0,0.0,0.0,0.0,294256.223941,459.776060
9,"Davidson County, Tennessee",715884.0,715884.0,400622.0,173092.0,3942.0,27869.0,354.0,56230.0,53775.0,386835.0,98119.0,328309.0,328309.0,301236.0,27073.0,24801.0,7230.0,4378.0,156.0,2497.0,199.0,17571.0,15112.0,0.0,2459.0,322364.394464,503.695144


In [294]:
total = counties.append(inc)
total = total.drop(columns = ['raceeth_total_series', 'raceeth_onerace', 'raceeth_nothispanicorlatino'])
total.head()

,NAME,pop,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,gq_total_series,gq_institutionalized,gq_institutionalized_correctionalfacilityadults,gq_institutionalized_juvenilefacility,gq_institutionalized_nursingorskillednursingfacility,gq_institutionalized_other,gq_noninstitutionalized,gq_noninstitutionalized_collegeoruniversitystudenthousing,gq_noninstitutionalized_militaryquarters,gq_noninstitutionalized_other,Square Acres,Square Miles
0,"Montgomery County, Tennessee",220069.0,137775.0,44569.0,1154.0,5202.0,1000.0,7890.0,22479.0,131294.0,22934.0,85714.0,85714.0,79841.0,5873.0,4714.0,1758.0,1084.0,32.0,642.0,0.0,2956.0,1450.0,1321.0,185.0,345067.747777,539.169188
1,"Rutherford County, Tennessee",341486.0,226227.0,53977.0,1739.0,12538.0,254.0,19755.0,26996.0,219194.0,38317.0,131216.0,131216.0,124465.0,6751.0,5723.0,1851.0,774.0,43.0,955.0,79.0,3872.0,2831.0,0.0,1041.0,396329.938251,619.266485
2,"Sumner County, Tennessee",196281.0,158075.0,15740.0,711.0,2956.0,119.0,5321.0,13359.0,155169.0,12870.0,78995.0,78995.0,74472.0,4523.0,1612.0,1377.0,572.0,25.0,780.0,0.0,235.0,0.0,0.0,235.0,338853.333043,529.459150
3,"Trousdale County, Tennessee",11615.0,8883.0,1922.0,30.0,27.0,0.0,205.0,548.0,8786.0,375.0,3749.0,3749.0,3469.0,280.0,2569.0,2549.0,2505.0,0.0,44.0,0.0,20.0,0.0,0.0,20.0,73178.560819,114.341678
4,"Williamson County, Tennessee",247726.0,203600.0,9876.0,574.0,12953.0,117.0,5172.0,15434.0,200408.0,14261.0,91133.0,91133.0,86884.0,4249.0,1359.0,1252.0,189.0,28.0,1035.0,0.0,107.0,0.0,0.0,107.0,373031.486909,582.862598


In [295]:
data = total

In [296]:
data = data.set_index('NAME').transpose()

In [297]:
data.head()

NAME,"Montgomery County, Tennessee","Rutherford County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee","Cheatham County, Tennessee","Robertson County, Tennessee","Stewart County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Maury County, Tennessee",Cheatham Incorporated,Davidson Incorporated,Dickson Incorporated,Houston Incorporated,Humphreys Incorporated,Maury Incorporated,Montgomery Incorporated,Robertson Incorporated,Rutherford Incorporated,Stewart Incorporated,Sumner Incorporated,Trousdale Incorporated,Williamson Incorporated,Wilson Incorporated
pop,220069.0,341486.0,196281.0,11615.0,247726.0,147737.0,41072.0,72803.0,13657.0,715884.0,54315.0,8283.0,18990.0,100974.0,14896.0,715884.0,23693.0,2556.0,7744.0,63122.0,166722.0,43499.0,251362.0,2422.0,147004.0,11615.0,192855.0,89599.0
raceeth_white_alone,137775.0,226227.0,158075.0,8883.0,203600.0,120782.0,36788.0,58467.0,12463.0,400622.0,47609.0,7557.0,17270.0,77837.0,13056.0,400622.0,19624.0,2292.0,6866.0,44410.0,94978.0,32060.0,152339.0,2204.0,114373.0,8883.0,154406.0,69622.0
raceeth_blackafricanamerican_alone,44569.0,53977.0,15740.0,1922.0,9876.0,10191.0,819.0,5168.0,172.0,173092.0,1962.0,210.0,506.0,11349.0,497.0,173092.0,1419.0,104.0,320.0,9967.0,40657.0,4604.0,48728.0,72.0,13978.0,1922.0,8555.0,8002.0
raceeth_americanindianalaskanative_alone,1154.0,1739.0,711.0,30.0,574.0,601.0,135.0,335.0,78.0,3942.0,242.0,47.0,43.0,417.0,43.0,3942.0,136.0,11.0,24.0,297.0,914.0,250.0,1416.0,5.0,562.0,30.0,434.0,395.0
raceeth_asian_alone,5202.0,12538.0,2956.0,27.0,12953.0,3106.0,191.0,460.0,77.0,27869.0,312.0,16.0,65.0,952.0,97.0,27869.0,200.0,4.0,38.0,803.0,4143.0,352.0,10459.0,10.0,2629.0,27.0,11976.0,2720.0


In [298]:
data['Montgomery Unincorporated'] = data['Montgomery County, Tennessee'] - data['Montgomery Incorporated']
data['Rutherford Unincorporated'] = data['Rutherford County, Tennessee'] - data['Rutherford Incorporated']
data['Sumner Unincorporated'] = data['Sumner County, Tennessee'] - data['Sumner Incorporated']
data['Trousdale Unincorporated'] = data['Trousdale County, Tennessee'] - data['Trousdale Incorporated']
data['Williamson Unincorporated'] = data['Williamson County, Tennessee'] - data['Williamson Incorporated']
data['Wilson Unincorporated'] = data['Wilson County, Tennessee'] - data['Wilson Incorporated']
data['Cheatham Unincorporated'] = data['Cheatham County, Tennessee'] - data['Cheatham Incorporated']
data['Robertson Unincorporated'] = data['Robertson County, Tennessee'] - data['Robertson  Incorporated']
data['Stewart Unincorporated'] = data['Stewart County, Tennessee'] - data['Stewart Incorporated']
data['Davidson Unincorporated'] = data['Davidson County, Tennessee'] - data['Davidson Incorporated']
data['Dickson Unincorporated'] = data['Dickson County, Tennessee'] - data['Dickson Incorporated']
data['Houston Unincorporated'] = data['Houston County, Tennessee'] - data['Houston Incorporated']
data['Humphreys Unincorporated'] = data['Humphreys County, Tennessee'] - data['Humphreys Incorporated']
data['Maury Unincorporated'] = data['Maury County, Tennessee'] - data['Maury Incorporated']
data = data.drop(columns = ['Montgomery Incorporated', 'Rutherford Incorporated', 'Sumner Incorporated', 'Trousdale Incorporated', 'Williamson Incorporated', 
                            'Wilson Incorporated', 'Cheatham Incorporated', 'Robertson  Incorporated', 'Stewart Incorporated', 'Davidson Incorporated', 
                            'Dickson Incorporated', 'Houston Incorporated', 'Humphreys Incorporated', 'Maury Incorporated'])

In [301]:
data = data.transpose()

In [302]:
data.head()

,pop,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,gq_total_series,gq_institutionalized,gq_institutionalized_correctionalfacilityadults,gq_institutionalized_juvenilefacility,gq_institutionalized_nursingorskillednursingfacility,gq_institutionalized_other,gq_noninstitutionalized,gq_noninstitutionalized_collegeoruniversitystudenthousing,gq_noninstitutionalized_militaryquarters,gq_noninstitutionalized_other,Square Acres,Square Miles
NAME,,,,,,,,,,,,,,,,,,,,,,,,,,
"Montgomery County, Tennessee",220069.0,137775.0,44569.0,1154.0,5202.0,1000.0,7890.0,22479.0,131294.0,22934.0,85714.0,85714.0,79841.0,5873.0,4714.0,1758.0,1084.0,32.0,642.0,0.0,2956.0,1450.0,1321.0,185.0,345067.747777,539.169188
"Rutherford County, Tennessee",341486.0,226227.0,53977.0,1739.0,12538.0,254.0,19755.0,26996.0,219194.0,38317.0,131216.0,131216.0,124465.0,6751.0,5723.0,1851.0,774.0,43.0,955.0,79.0,3872.0,2831.0,0.0,1041.0,396329.938251,619.266485
"Sumner County, Tennessee",196281.0,158075.0,15740.0,711.0,2956.0,119.0,5321.0,13359.0,155169.0,12870.0,78995.0,78995.0,74472.0,4523.0,1612.0,1377.0,572.0,25.0,780.0,0.0,235.0,0.0,0.0,235.0,338853.333043,529.459150
"Trousdale County, Tennessee",11615.0,8883.0,1922.0,30.0,27.0,0.0,205.0,548.0,8786.0,375.0,3749.0,3749.0,3469.0,280.0,2569.0,2549.0,2505.0,0.0,44.0,0.0,20.0,0.0,0.0,20.0,73178.560819,114.341678
"Williamson County, Tennessee",247726.0,203600.0,9876.0,574.0,12953.0,117.0,5172.0,15434.0,200408.0,14261.0,91133.0,91133.0,86884.0,4249.0,1359.0,1252.0,189.0,28.0,1035.0,0.0,107.0,0.0,0.0,107.0,373031.486909,582.862598


In [303]:
#population
data['Population'] = data['pop']
#race and ethnicity
#White Alone
data['White Alone'] = data['raceeth_white_alone']
data['White Alone %'] = percent(data['White Alone'], data['Population'])
#Population not White
data['Non-White Population: Race'] = data['Population'] - data['White Alone']
#Black or African American Alone 
data['Black or African American Alone'] = data['raceeth_blackafricanamerican_alone']
data['Black or African American Alone %'] = percent(data['Black or African American Alone'], data['Population'])
data['Black or African American Alone % Non-White Population'] = percent(data['Black or African American Alone'], data['Non-White Population: Race'])
#American Indian and Alaska Native Alone
data['American Indian Alaska Native Alone'] = data['raceeth_americanindianalaskanative_alone']
data['American Indian Alaska Native Alone %'] = percent(data['American Indian Alaska Native Alone'], data['Population'])
data['American Indian Alaska Native Alone % Non-White Population'] = percent(data['American Indian Alaska Native Alone'], data['Non-White Population: Race'])
#Asian Alone
data['Asian Alone'] = data['raceeth_asian_alone']
data['Asian Alone %'] = percent(data['Asian Alone'], data['Population'])
data['Asian Alone % Non-White Population'] = percent(data['Asian Alone'], data['Non-White Population: Race'])
#Native Hawaiian Other Pacific Islander Alone
data['Native Hawaiian Other Pacific Islander Alone'] = data['raceeth_nativehawaiianotherpacificislander_alone']
data['Native Hawaiian Other Pacific Islander Alone %'] = percent(data['Native Hawaiian Other Pacific Islander Alone'], data['Population'])
data['Native Hawaiian Other Pacific Islander Alone % Non-White Population'] = percent(data['Native Hawaiian Other Pacific Islander Alone'], data['Non-White Population: Race'])
#Some Other Race Alone
data['Some Other Race Alone'] = data['raceeth_someotherrace_alone']
data['Some Other Race Alone %'] = percent(data['Some Other Race Alone'], data['Population'])
data['Some Other Race Alone % Non-White Population'] = percent(data['Some Other Race Alone'], data['Non-White Population: Race'])
#Two or More Races
data['Two or More Races'] = data['raceeth_twoormoreraces']
data['Two or More Races %'] = percent(data['Two or More Races'], data['Population'])
data['Two or More Races % Non-White Population'] = percent(data['Two or More Races'], data['Non-White Population: Race'])
#Hispanic or Latino
data['Hispanic or Latino'] = data['raceeth_hispanicorlatino']
data['Hispanic or Latino %'] = percent(data['Hispanic or Latino'], data['Population'])
#not hispanic or latino
data['Not Hispanic or Latino'] = data['Population'] - data['Hispanic or Latino']
data['Not Hispanic or Latino %'] = percent(data['Not Hispanic or Latino'], data['Population'])
#Data Minority
data['Minority'] = data['Population'] - data['raceeth_whitealone_nothispanicorlatino']
data['Minority %'] = percent(data['Minority'], data['Population'])
#not minority
data['White Alone Not Hispanic or Latino'] = data['raceeth_whitealone_nothispanicorlatino']
data['White Alone Not Hispanic or Latino %'] = percent(data['White Alone Not Hispanic or Latino'], data['Population'])
#occupancy
data['Occupancy:Total Households'] = data['units_allhousing']
data['Occupancy:Occupied Units'] = data['occupancy_occupiedunits']
data['Occupancy %:Occupied Units'] = percent(data['Occupancy:Occupied Units'], data['Occupancy:Total Households'])
data['Occupancy:Vacant Units'] = data['occupancy_vacantunits']
data['Occupancy %:Vacant Units'] = percent(data['Occupancy:Vacant Units'], data['Occupancy:Total Households'])
#group quarters
data['Group Quarters:Population'] = data['gq_total_series']
data['Group Quarters %:Population'] = percent(data['gq_total_series'], data['Population'])
data['Group Quarters:Institutionalized'] = data['gq_institutionalized']
data['Group Quarters %:Institutionalized'] = percent(data['Group Quarters:Institutionalized'], data['Group Quarters:Population'])
data['Group Quarters:Institutionalized, Adults in Correctional Facility'] = data['gq_institutionalized_correctionalfacilityadults']
data['Group Quarters % Institutionalized:Adults in Correctional Facility'] = percent(data['Group Quarters:Institutionalized, Adults in Correctional Facility'], 
                                                                                     data['Group Quarters:Institutionalized'])
data['Group Quarters %:Institutionalized Adults in Correctional Facility'] = percent(data['Group Quarters:Institutionalized, Adults in Correctional Facility'], 
                                                                                     data['Group Quarters:Population'])
data['Group Quarters:Institutionalized, Juvenile Facility'] = data['gq_institutionalized_juvenilefacility']
data['Group Quarters % Institutionalized:Juvenile Facility'] = percent(data['Group Quarters:Institutionalized, Juvenile Facility'], 
                                                                        data['Group Quarters:Institutionalized'])
data['Group Quarters %:Institutionalized in Juvenile Facility'] = percent(data['Group Quarters:Institutionalized, Juvenile Facility'], 
                                                                           data['Group Quarters:Population'])
data['Group Quarters:Institutionalized, Nursing or Skilled Nursing Facility'] = data['gq_institutionalized_nursingorskillednursingfacility']
data['Group Quarters % Institutionalized:Nursing or Skilled Nursing Facility'] = percent(data['Group Quarters:Institutionalized, Nursing or Skilled Nursing Facility'], 
                                                                        data['Group Quarters:Institutionalized'])
data['Group Quarters %:Institutionalized in Nursing or Skilled Nursing Facility'] = percent(data['Group Quarters:Institutionalized, Nursing or Skilled Nursing Facility'], 
                                                                           data['Group Quarters:Population'])
data['Group Quarters:Institutionalized, Other Facility'] = data['gq_institutionalized_other']
data['Group Quarters % Institutionalized:Other Facility'] = percent(data['Group Quarters:Institutionalized, Other Facility'], 
                                                                     data['Group Quarters:Institutionalized'])
data['Group Quarters %:Institutionalized in Other Facility'] = percent(data['Group Quarters:Institutionalized, Other Facility'], 
                                                                        data['Group Quarters:Population'])
data['Group Quarters:Noninstitutionalized'] = data['gq_noninstitutionalized']
data['Group Quarters %:Noninstitutionalized'] = percent(data['Group Quarters:Institutionalized, Other Facility'], 
                                                                     data['Group Quarters:Population'])
data['Group Quarters:Noninstitutionalized, College or University Student Housing'] = data['gq_noninstitutionalized_collegeoruniversitystudenthousing']
data['Group Quarters % Noninstitutionalized:College or University Student Housing'] = percent(data['Group Quarters:Noninstitutionalized, College or University Student Housing'], 
                                                                     data['Group Quarters:Noninstitutionalized'])
data['Group Quarters %:Noninstitutionalized in College or University Student Housing'] = percent(data['Group Quarters:Noninstitutionalized, College or University Student Housing'], 
                                                                        data['Group Quarters:Population'])
data['Group Quarters:Noninstitutionalized, Military Quarters'] = data['gq_noninstitutionalized_militaryquarters']
data['Group Quarters % Noninstitutionalized:Military Quarters'] = percent(data['Group Quarters:Noninstitutionalized, Military Quarters'], 
                                                                     data['Group Quarters:Noninstitutionalized'])
data['Group Quarters %:Noninstitutionalized in Military Quarters'] = percent(data['Group Quarters:Noninstitutionalized, Military Quarters'], 
                                                                        data['Group Quarters:Population'])
data['Group Quarters:Noninstitutionalized, Other'] = data['gq_noninstitutionalized_militaryquarters']
data['Group Quarters % Noninstitutionalized:Other'] = percent(data['Group Quarters:Noninstitutionalized, Military Quarters'], 
                                                                     data['Group Quarters:Noninstitutionalized'])
data['Group Quarters %:Noninstitutionalized in Other'] = percent(data['Group Quarters:Noninstitutionalized, Military Quarters'], 
                                                                        data['Group Quarters:Population'])
data['Population Per Square Mile'] = data['Population']/data['Square Miles']
data['Housing Units Per Square Mile'] = data['Occupancy:Total Households']/data['Square Miles']
data['Population Per Square Acre'] = data['Population']/data['Square Acres']
data['Housing Units Per Square Acre'] = data['Occupancy:Total Households']/data['Square Acres']

In [304]:
data = data[['Population', 'White Alone', 'White Alone %', 'Non-White Population: Race', 'Black or African American Alone', 
             'Black or African American Alone %', 'Black or African American Alone % Non-White Population', 
             'American Indian Alaska Native Alone', 'American Indian Alaska Native Alone %', 'American Indian Alaska Native Alone % Non-White Population', 
             'Asian Alone', 'Asian Alone %', 'Asian Alone % Non-White Population', 'Native Hawaiian Other Pacific Islander Alone', 
             'Native Hawaiian Other Pacific Islander Alone %', 'Native Hawaiian Other Pacific Islander Alone % Non-White Population', 'Some Other Race Alone', 
             'Some Other Race Alone %', 'Some Other Race Alone % Non-White Population', 'Two or More Races', 'Two or More Races %', 
             'Two or More Races % Non-White Population', 'Hispanic or Latino', 'Hispanic or Latino %', 'Not Hispanic or Latino', 
             'Not Hispanic or Latino %', 'Minority', 'Minority %', 'White Alone Not Hispanic or Latino', 'White Alone Not Hispanic or Latino %', 
             'Occupancy:Total Households', 'Occupancy:Occupied Units', 'Occupancy %:Occupied Units', 'Occupancy:Vacant Units', 'Occupancy %:Vacant Units', 
             'Group Quarters:Population', 'Group Quarters %:Population', 'Group Quarters:Institutionalized', 'Group Quarters %:Institutionalized', 
             'Group Quarters:Institutionalized, Adults in Correctional Facility', 'Group Quarters % Institutionalized:Adults in Correctional Facility', 
             'Group Quarters %:Institutionalized Adults in Correctional Facility', 'Group Quarters:Institutionalized, Juvenile Facility', 
             'Group Quarters % Institutionalized:Juvenile Facility', 'Group Quarters %:Institutionalized in Juvenile Facility', 
             'Group Quarters:Institutionalized, Nursing or Skilled Nursing Facility', 'Group Quarters % Institutionalized:Nursing or Skilled Nursing Facility', 
             'Group Quarters %:Institutionalized in Nursing or Skilled Nursing Facility', 'Group Quarters:Institutionalized, Other Facility', 
             'Group Quarters % Institutionalized:Other Facility', 'Group Quarters %:Institutionalized in Other Facility', 
             'Group Quarters:Noninstitutionalized', 'Group Quarters %:Noninstitutionalized', 
             'Group Quarters:Noninstitutionalized, College or University Student Housing', 
             'Group Quarters % Noninstitutionalized:College or University Student Housing', 
             'Group Quarters %:Noninstitutionalized in College or University Student Housing', 
             'Group Quarters:Noninstitutionalized, Military Quarters', 'Group Quarters % Noninstitutionalized:Military Quarters', 
             'Group Quarters %:Noninstitutionalized in Military Quarters', 'Group Quarters:Noninstitutionalized, Other', 
             'Group Quarters % Noninstitutionalized:Other', 'Group Quarters %:Noninstitutionalized in Other', 'Square Miles', 
             'Square Acres', 'Population Per Square Mile', 'Population Per Square Acre', 'Housing Units Per Square Mile', 
             'Housing Units Per Square Acre']]

In [306]:
data = data.reset_index()
data.head()

,NAME,Population,White Alone,White Alone %,Non-White Population: Race,Black or African American Alone,Black or African American Alone %,Black or African American Alone % Non-White Population,American Indian Alaska Native Alone,American Indian Alaska Native Alone %,American Indian Alaska Native Alone % Non-White Population,Asian Alone,Asian Alone %,Asian Alone % Non-White Population,Native Hawaiian Other Pacific Islander Alone,Native Hawaiian Other Pacific Islander Alone %,Native Hawaiian Other Pacific Islander Alone % Non-White Population,Some Other Race Alone,Some Other Race Alone %,Some Other Race Alone % Non-White Population,Two or More Races,Two or More Races %,Two or More Races % Non-White Population,Hispanic or Latino,Hispanic or Latino %,Not Hispanic or Latino,Not Hispanic or Latino %,Minority,Minority %,White Alone Not Hispanic or Latino,White Alone Not Hispanic or Latino %,Occupancy:Total Households,Occupancy:Occupied Units,Occupancy %:Occupied Units,Occupancy:Vacant Units,Occupancy %:Vacant Units,Group Quarters:Population,Group Quarters %:Population,Group Quarters:Institutionalized,Group Quarters %:Institutionalized,"Group Quarters:Institutionalized, Adults in Correctional Facility",Group Quarters % Institutionalized:Adults in Correctional Facility,Group Quarters %:Institutionalized Adults in Correctional Facility,"Group Quarters:Institutionalized, Juvenile Facility",Group Quarters % Institutionalized:Juvenile Facility,Group Quarters %:Institutionalized in Juvenile Facility,"Group Quarters:Institutionalized, Nursing or Skilled Nursing Facility",Group Quarters % Institutionalized:Nursing or Skilled Nursing Facility,Group Quarters %:Institutionalized in Nursing or Skilled Nursing Facility,"Group Quarters:Institutionalized, Other Facility",Group Quarters % Institutionalized:Other Facility,Group Quarters %:Institutionalized in Other Facility,Group Quarters:Noninstitutionalized,Group Quarters %:Noninstitutionalized,"Group Quarters:Noninstitutionalized, College or University Student Housing",Group Quarters % Noninstitutionalized:College or University Student Housing,Group Quarters %:Noninstitutionalized in College or University Student Housing,"Group Quarters:Noninstitutionalized, Military Quarters",Group Quarters % Noninstitutionalized:Military Quarters,Group Quarters %:Noninstitutionalized in Military Quarters,"Group Quarters:Noninstitutionalized, Other",Group Quarters % Noninstitutionalized:Other,Group Quarters %:Noninstitutionalized in Other,Square Miles,Square Acres,Population Per Square Mile,Population Per Square Acre,Housing Units Per Square Mile,Housing Units Per Square Acre
0,"Montgomery County, Tennessee",220069.0,137775.0,62.605365,82294.0,44569.0,20.252285,54.158262,1154.0,0.524381,1.402289,5202.0,2.363804,6.321238,1000.0,0.454403,1.215155,7890.0,3.585239,9.587576,22479.0,10.214524,27.315479,22934.0,10.421277,197135.0,89.578723,88775.0,40.339621,131294.0,59.660379,85714.0,79841.0,93.148144,5873.0,6.851856,4714.0,2.142055,1758.0,37.293169,1084.0,61.660978,22.995333,32.0,1.820250,0.678829,642.0,36.518771,13.619007,0.0,0.000000,0.000000,2956.0,0.000000,1450.0,49.052774,30.759440,1321.0,44.688769,28.02291,1321.0,44.688769,28.02291,539.169188,345067.747777,408.163161,0.637756,158.974218,0.248398
1,"Rutherford County, Tennessee",341486.0,226227.0,66.247811,115259.0,53977.0,15.806505,46.831050,1739.0,0.509245,1.508776,12538.0,3.671600,10.878109,254.0,0.074381,0.220373,19755.0,5.785010,17.139659,26996.0,7.905449,23.422032,38317.0,11.220665,303169.0,88.779335,122292.0,35.811717,219194.0,64.188283,131216.0,124465.0,94.855048,6751.0,5.144952,5723.0,1.675911,1851.0,32.343177,774.0,41.815235,13.524375,43.0,2.323069,0.751354,955.0,51.593733,16.687052,79.0,4.267963,1.380395,3872.0,1.380395,2831.0,73.114669,49.467063,0.0,0.000000,0.00000,0.0,0.000000,0.00000,619.266485,396329.938251,551.436269,0.861621,211.889394,0.331078
2,"Sumner County, Tennessee",196281.0,158075.0,80.535049,38206.0,15740.0,8.019115,41.197718,711.0,0.362236

In [310]:
data['Population Rank'] = data['Population'].rank(ascending = False)
# data = data.sort_values('Population', ascending = False).reset_index()
data.head()

,NAME,Population,White Alone,White Alone %,Non-White Population: Race,Black or African American Alone,Black or African American Alone %,Black or African American Alone % Non-White Population,American Indian Alaska Native Alone,American Indian Alaska Native Alone %,American Indian Alaska Native Alone % Non-White Population,Asian Alone,Asian Alone %,Asian Alone % Non-White Population,Native Hawaiian Other Pacific Islander Alone,Native Hawaiian Other Pacific Islander Alone %,Native Hawaiian Other Pacific Islander Alone % Non-White Population,Some Other Race Alone,Some Other Race Alone %,Some Other Race Alone % Non-White Population,Two or More Races,Two or More Races %,Two or More Races % Non-White Population,Hispanic or Latino,Hispanic or Latino %,Not Hispanic or Latino,Not Hispanic or Latino %,Minority,Minority %,White Alone Not Hispanic or Latino,White Alone Not Hispanic or Latino %,Occupancy:Total Households,Occupancy:Occupied Units,Occupancy %:Occupied Units,Occupancy:Vacant Units,Occupancy %:Vacant Units,Group Quarters:Population,Group Quarters %:Population,Group Quarters:Institutionalized,Group Quarters %:Institutionalized,"Group Quarters:Institutionalized, Adults in Correctional Facility",Group Quarters % Institutionalized:Adults in Correctional Facility,Group Quarters %:Institutionalized Adults in Correctional Facility,"Group Quarters:Institutionalized, Juvenile Facility",Group Quarters % Institutionalized:Juvenile Facility,Group Quarters %:Institutionalized in Juvenile Facility,"Group Quarters:Institutionalized, Nursing or Skilled Nursing Facility",Group Quarters % Institutionalized:Nursing or Skilled Nursing Facility,Group Quarters %:Institutionalized in Nursing or Skilled Nursing Facility,"Group Quarters:Institutionalized, Other Facility",Group Quarters % Institutionalized:Other Facility,Group Quarters %:Institutionalized in Other Facility,Group Quarters:Noninstitutionalized,Group Quarters %:Noninstitutionalized,"Group Quarters:Noninstitutionalized, College or University Student Housing",Group Quarters % Noninstitutionalized:College or University Student Housing,Group Quarters %:Noninstitutionalized in College or University Student Housing,"Group Quarters:Noninstitutionalized, Military Quarters",Group Quarters % Noninstitutionalized:Military Quarters,Group Quarters %:Noninstitutionalized in Military Quarters,"Group Quarters:Noninstitutionalized, Other",Group Quarters % Noninstitutionalized:Other,Group Quarters %:Noninstitutionalized in Other,Square Miles,Square Acres,Population Per Square Mile,Population Per Square Acre,Housing Units Per Square Mile,Housing Units Per Square Acre,Population Rank
9,"Davidson County, Tennessee",715884.0,400622.0,55.961860,315262.0,173092.0,24.178778,54.904175,3942.0,0.550648,1.250389,27869.0,3.892949,8.839949,354.0,0.049449,0.112288,56230.0,7.854624,17.835959,53775.0,7.511692,17.057241,98119.0,13.705991,617765.0,86.294009,329049.0,45.964011,386835.0,54.035989,328309.0,301236.0,91.753805,27073.0,8.246195,24801.0,3.464388,7230.0,29.152050,4378.0,60.553250,17.652514,156.0,2.157676,0.629007,2497.0,34.536653,10.068142,199.0,2.752420,0.802387,17571.0,0.802387,15112.0,86.005350,60.933027,0.0,0.000000,0.00000,0.0,0.000000,0.00000,503.695144,322364.394464,1421.264447,2.220729,651.801003,1.018441,1.0
1,"Rutherford County, Tennessee",341486.0,226227.0,66.247811,115259.0,53977.0,15.806505,46.831050,1739.0,0.509245,1.508776,12538.0,3.671600,10.878109,254.0,0.074381,0.220373,19755.0,5.785010,17.139659,26996.0,7.905449,23.422032,38317.0,11.220665,303169.0,88.779335,122292.0,35.811717,219194.0,64.188283,131216.0,124465.0,94.855048,6751.0,5.144952,5723.0,1.675911,1851.0,32.343177,774.0,41.815235,13.524375,43.0,2.323069,0.751354,955.0,51.593733,16.687052,79.0,4.267963,1.380395,3872.0,1.380395,2831.0,73.114669,49.467063,0.0,0.000000,0.00000,0.0,0.000000,0.00000,619.266485,396329.938251,551.436269,0.861621,211.889394,0.331078,2.0
4,"Williamson County, Tennessee",247726.0,203600.0,82.187578,44126.0,9876.0,3.9

In [311]:
data.to_csv('../../PL SPECIAL/countyandunincorporated.csv', index = False)